In [1]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error
import os
from dotenv import load_dotenv

load_dotenv("../.env")

PG_HOST = os.getenv('POSTGRES_HOST')
PG_PORT = os.getenv('POSTGRES_PORT')
PG_PASSWORD = os.getenv('POSTGRES_PASSWORD')
PG_USER = os.getenv('POSTGRES_USER')
PG_DB = os.getenv('POSTGRES_DB')

# Read in the data file that we need to insert
df = pd.read_excel("./Data/Cleaned_Root_letters.xlsx")

# Clean the data right at the start 
df = df.convert_dtypes()
df['ARABIC'] = df['ARABIC'].str.strip()
df['Root_Letters'] = df['Root_Letters'].str.strip()

# Prep Data for RootWord Table 
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

# Turn it into string and get rid of white spaces 
df_roots = df_roots.convert_dtypes()
df_roots['Root_Words'] = df_roots['Root_Words'].str.strip()

# Put the data in the format the db accepts
root_data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    root_data.append((row, arabic_text))

# Prep Data for the ArabicWord Table
arabic_df = df.drop(['ID', 'Transliteration'], axis=1)
arabic_df = arabic_df.drop_duplicates()
arabic_df["primary_key"] = list(range(2000,19623)) 
arabic_df = arabic_df.convert_dtypes()

# Merge the df to get the RootWords PK (called "index")
arabic_words_df = arabic_df.merge(df_roots, how='inner', left_on='Root_Letters', right_on='Root_Words')
arabic_words_df = arabic_words_df.drop(['Root_Letters', 'Root_Words'], axis=1)

# Rename the index column to root_id to make it clear
arabic_words_df.rename(columns = {'index':'root_id'}, inplace = True)

# Put the dataframe into the format for postgres sql insertion 
arabic_words_data = []
for index, row in arabic_words_df.iterrows():
    arabic_words_data.append( (row['primary_key'], row['ARABIC'], row['root_id']) )

In [10]:
all_words

[['بِسْمِ', 'اللَّهِ', 'الرَّحْمَـٰنِ', 'الرَّحِيمِ'],
 ['الْحَمْدُ', 'لِلَّهِ', 'رَبِّ', 'الْعَالَمِينَ'],
 ['الرَّحْمَـٰنِ', 'الرَّحِيمِ'],
 ['مَالِكِ', 'يَوْمِ', 'الدِّينِ'],
 ['إِيَّاكَ', 'نَعْبُدُ', 'وَإِيَّاكَ', 'نَسْتَعِينُ'],
 ['اهْدِنَا', 'الصِّرَاطَ', 'الْمُسْتَقِيمَ'],
 ['صِرَاطَ',
  'الَّذِينَ',
  'أَنْعَمْتَ',
  'عَلَيْهِمْ',
  'غَيْرِ',
  'الْمَغْضُوبِ',
  'عَلَيْهِمْ',
  'وَلَا',
  'الضَّالِّينَ'],
 ['بِسْمِ', 'اللَّهِ', 'الرَّحْمَـٰنِ', 'الرَّحِيمِ', 'الم'],
 ['ذَٰلِكَ',
  'الْكِتَابُ',
  'لَا',
  'رَيْبَ',
  'ۛ',
  'فِيهِ',
  'ۛ',
  'هُدًى',
  'لِّلْمُتَّقِينَ'],
 ['الَّذِينَ',
  'يُؤْمِنُونَ',
  'بِالْغَيْبِ',
  'وَيُقِيمُونَ',
  'الصَّلَاةَ',
  'وَمِمَّا',
  'رَزَقْنَاهُمْ',
  'يُنفِقُونَ'],
 ['وَالَّذِينَ',
  'يُؤْمِنُونَ',
  'بِمَا',
  'أُنزِلَ',
  'إِلَيْكَ',
  'وَمَا',
  'أُنزِلَ',
  'مِن',
  'قَبْلِكَ',
  'وَبِالْآخِرَةِ',
  'هُمْ',
  'يُوقِنُونَ'],
 ['أُولَـٰئِكَ',
  'عَلَىٰ',
  'هُدًى',
  'مِّن',
  'رَّبِّهِمْ',
  'ۖ',
  'وَأُولَـٰئِكَ',
  'هُمُ',
  'الْمُفْلِ

In [11]:
unpacked_lst = []
for word in all_words:
    if len(word) > 1:
        for words in word:
            unpacked_lst.append(words)
    else:
        unpacked_lst.append(word)       

In [13]:
len(unpacked_lst) # Corpus Quran / SQL dump

82627

In [18]:
unique_words

{'اتَّبَعَنِ',
 'نَوْمَكُمْ',
 'الْعِجْلَ',
 'فَوَيْلٌ',
 'مُسْلِمَيْنِ',
 'مَاذَا',
 'تُكَذِّبُوا',
 'الرَّحْمَةَ',
 'وَنَزَعْنَا',
 'الشِّرْكَ',
 'وَلْيَتَّقِ',
 'حَاجِزِينَ',
 'أَكْثَرَهُمْ',
 'إِلَـٰهُكُمْ',
 'تَوَلَّيْتُمْ',
 'يُؤْتِيهِ',
 'عَيْنِي',
 'يُعَقِّبْ',
 'تُصْبِحُونَ',
 'شُرَكَاءَ',
 'وَحَلَالًا',
 'فَوْقَهَا',
 'أَأَقْرَرْتُمْ',
 'الْمُلْقِينَ',
 'بِرُسُلٍ',
 'أَعْرِضْ',
 'الْمَاءِ',
 'وَأَيْدِيَكُمْ',
 'جَهُولًا',
 'أَنظُرْ',
 'سُكِّرَتْ',
 'فَأُنَبِّئُكُم',
 'لِّتَعْتَدُوا',
 'يُتْبِعُونَ',
 'لَّبِثْتُمْ',
 'وَيُكَفِّرَ',
 'مَالِكُونَ',
 'بَيَّنَّاهُ',
 'فَالْحُكْمُ',
 'وَقُودُهَا',
 'فَلْيُؤَدِّ',
 'ذِكْرٌ',
 'كَدُعَاءِ',
 'بُيُوتًا',
 'وَّنَصَرُوا',
 'وَقُتِّلُوا',
 'تَرَكْتُ',
 'يَجِدِ',
 'لَّمِنَ',
 'وَبَثَّ',
 'أَضْلَلْتُمْ',
 'يُضْلِلْ',
 'مُخْلِصُونَ',
 'قَلْبٌ',
 'يَتَذَكَّرُ',
 'مِهَادًا',
 'وَنُرِيدُ',
 'فَانتَشِرُوا',
 'خَفِيٍّ',
 'أَيُّكُمْ',
 'وَنِسَاءَنَا',
 'صَاحِبَيِ',
 'ثَيِّبَاتٍ',
 'عَصَوْنِي',
 'خَوْفٌ',
 'وَالطَّارِقِ',
 'وَأَحَاطَتْ',
 'نَفْسٍ',

In [14]:
df # Excel sheet / Online 

,ID,ARABIC,Transliteration,Root_Letters
0,1:1,بِسْمِ,bis'mi,س م و
1,1:1,اللَّهِ,l-lahi,ا ل ه
2,1:1,الرَّحْمَنِ,l-raḥmāni,ر ح م
3,1:1,الرَّحِيمِ,l-raḥīmi,ر ح م
4,1:2,الْحَمْدُ,al-ḥamdu,ح م د
...,...,...,...,...
77426,114:5,صُدُورِ,ṣudūri,ص د ر
77427,114:5,النَّاسِ,l-nāsi,ن و س
77428,114:6,مِنَ,mina,م ن
77429,114:6,الْجِنَّةِ,l-jinati,ج ن ن


In [68]:
len(unpacked_lst)

82627

In [64]:
all_words2 = list(flatten(all_words))

NameError: name 'flatten' is not defined

In [59]:
all_words2

[['بِسْمِ', 'اللَّهِ', 'الرَّحْمَـٰنِ', 'الرَّحِيمِ'],
 ['الْحَمْدُ', 'لِلَّهِ', 'رَبِّ', 'الْعَالَمِينَ'],
 ['الرَّحْمَـٰنِ', 'الرَّحِيمِ'],
 ['مَالِكِ', 'يَوْمِ', 'الدِّينِ'],
 ['إِيَّاكَ', 'نَعْبُدُ', 'وَإِيَّاكَ', 'نَسْتَعِينُ'],
 ['اهْدِنَا', 'الصِّرَاطَ', 'الْمُسْتَقِيمَ'],
 ['صِرَاطَ',
  'الَّذِينَ',
  'أَنْعَمْتَ',
  'عَلَيْهِمْ',
  'غَيْرِ',
  'الْمَغْضُوبِ',
  'عَلَيْهِمْ',
  'وَلَا',
  'الضَّالِّينَ'],
 ['بِسْمِ', 'اللَّهِ', 'الرَّحْمَـٰنِ', 'الرَّحِيمِ', 'الم'],
 ['ذَٰلِكَ',
  'الْكِتَابُ',
  'لَا',
  'رَيْبَ',
  'ۛ',
  'فِيهِ',
  'ۛ',
  'هُدًى',
  'لِّلْمُتَّقِينَ'],
 ['الَّذِينَ',
  'يُؤْمِنُونَ',
  'بِالْغَيْبِ',
  'وَيُقِيمُونَ',
  'الصَّلَاةَ',
  'وَمِمَّا',
  'رَزَقْنَاهُمْ',
  'يُنفِقُونَ'],
 ['وَالَّذِينَ',
  'يُؤْمِنُونَ',
  'بِمَا',
  'أُنزِلَ',
  'إِلَيْكَ',
  'وَمَا',
  'أُنزِلَ',
  'مِن',
  'قَبْلِكَ',
  'وَبِالْآخِرَةِ',
  'هُمْ',
  'يُوقِنُونَ'],
 ['أُولَـٰئِكَ',
  'عَلَىٰ',
  'هُدًى',
  'مِّن',
  'رَّبِّهِمْ',
  'ۖ',
  'وَأُولَـٰئِكَ',
  'هُمُ',
  'الْمُفْلِ

In [31]:
quran_df

,index,sura,aya,text
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ
3,4,1,4,مَالِكِ يَوْمِ الدِّينِ
4,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...,...
6231,6232,114,2,مَلِكِ النَّاسِ
6232,6233,114,3,إِلَـٰهِ النَّاسِ
6233,6234,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
6234,6235,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ


In [47]:
quran_df

,index,sura,aya,text
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ
3,4,1,4,مَالِكِ يَوْمِ الدِّينِ
4,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...,...
6231,6232,114,2,مَلِكِ النَّاسِ
6232,6233,114,3,إِلَـٰهِ النَّاسِ
6233,6234,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
6234,6235,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ


In [30]:
df

,ID,ARABIC,Transliteration,Root_Letters,Surah,Ayah,index_id
0,1:1,بِسْمِ,bis'mi,س م و,1,1,1
1,1:1,اللَّهِ,l-lahi,ا ل ه,1,1,1
2,1:1,الرَّحْمَنِ,l-raḥmāni,ر ح م,1,1,1
3,1:1,الرَّحِيمِ,l-raḥīmi,ر ح م,1,1,1
4,1:2,الْحَمْدُ,al-ḥamdu,ح م د,1,2,2
...,...,...,...,...,...,...,...
77426,114:5,صُدُورِ,ṣudūri,ص د ر,114,5,6235
77427,114:5,النَّاسِ,l-nāsi,ن و س,114,5,6235
77428,114:6,مِنَ,mina,م ن,114,6,6236
77429,114:6,الْجِنَّةِ,l-jinati,ج ن ن,114,6,6236


In [29]:
df_roots

,index,Root_Words
0,0,س ت ت
1,1,ن ط ف
2,2,ص ف ح
3,3,ه ي ا
4,4,ن ط ح
...,...,...
1794,1794,م س س
1795,1795,غ م ض
1796,1796,ل ح د
1797,1797,ح ف ي


In [28]:
arabic_words_df

,ARABIC,primary_key,root_id
0,بِسْمِ,2000,176
1,السَّمَاءِ,2158,176
2,وَالسَّمَاءَ,2199,176
3,سَمَاوَاتٍ,2302,176
4,الْأَسْمَاءَ,2325,176
...,...,...,...
17618,مَسَدٍ,19606,529
17619,الصَّمَدُ,19607,1317
17620,كُفُوًا,19610,901
17621,وَقَبَ,19614,188


In [19]:
arabic_words_df

,ARABIC,primary_key,root_id
0,بِسْمِ,2000,1329
1,السَّمَاءِ,2158,1329
2,وَالسَّمَاءَ,2199,1329
3,سَمَاوَاتٍ,2302,1329
4,الْأَسْمَاءَ,2325,1329
...,...,...,...
17618,مَسَدٍ,19606,1386
17619,الصَّمَدُ,19607,1215
17620,كُفُوًا,19610,439
17621,وَقَبَ,19614,1376


In [21]:
words_from_excel = list(arabic_words_df['ARABIC'])

In [22]:
words_from_excel

['بِسْمِ',
 'السَّمَاءِ',
 'وَالسَّمَاءَ',
 'سَمَاوَاتٍ',
 'الْأَسْمَاءَ',
 'بِأَسْمَاءِ',
 'بِأَسْمَائِهِمْ',
 'السَّمَاوَاتِ',
 'اسْمُهُ',
 'مُسَمًّى',
 'سَمَّيْتُهَا',
 'السَّمَاوَاتُ',
 'اسْمَ',
 'السَّمَاءَ',
 'اسْمُ',
 'أَسْمَاءٍ',
 'سَمَّيْتُمُوهَا',
 'الْأَسْمَاءُ',
 'أَسْمَائِهِ',
 'وَيَاسَمَاءُ',
 'أَسْمَاءً',
 'سَمُّوهُمْ',
 'وَالسَّمَاوَاتُ',
 'سَمِيًّا',
 'وَالسَّمَاوَاتِ',
 'سَمَّاكُمُ',
 'السَّمَاءُ',
 'سَمَاءٍ',
 'الِاسْمُ',
 'وَالسَّمَاءِ',
 'أَسْمَاءٌ',
 'لَيُسَمُّونَ',
 'تَسْمِيَةَ',
 'بِاسْمِ',
 'تُسَمَّى',
 'اللَّهِ',
 'لِلَّهِ',
 'اللَّهُ',
 'بِاللَّهِ',
 'اللَّهَ',
 'وَاللَّهُ',
 'فَاللَّهُ',
 'وَلِلَّهِ',
 'إِلَهَكَ',
 'وَإِلَهَ',
 'إِلَهًا',
 'وَإِلَهُكُمْ',
 'إِلَهٌ',
 'إِلَهَ',
 'اللَّهُمَّ',
 'إِلَهٍ',
 'إِلَهَيْنِ',
 'ءَالِهَةً',
 'وَاللَّهِ',
 'فَلِلَّهِ',
 'وَءَالِهَتَكَ',
 'ءَالِهَةٌ',
 'أَبِاللَّهِ',
 'ءَاللَّهُ',
 'ءَالِهَتِنَا',
 'ءَالِهَتُهُمُ',
 'تَاللَّهِ',
 'إِلَهُكُمْ',
 'ءَالِهَتِي',
 'وَإِلَهُ',
 'إِلَهِكَ',
 'إِلَهُكُمُ',
 'ءَالِهَتَكُمْ',
 'و

In [23]:
count = 0
for word in unique_words:
    if word in words_from_excel:
        #print(word)
        count +=1 

In [24]:
count

15152

In [25]:
arabic_df

,ARABIC,Root_Letters,primary_key
0,بِسْمِ,س م و,2000
1,اللَّهِ,ا ل ه,2001
2,الرَّحْمَنِ,ر ح م,2002
3,الرَّحِيمِ,ر ح م,2003
4,الْحَمْدُ,ح م د,2004
...,...,...,...
77410,حَسَدَ,ح س د,19618
77415,مَلِكِ,م ل ك,19619
77421,الْوَسْوَاسِ,و س و س,19620
77422,الْخَنَّاسِ,خ ن س,19621


In [41]:
quran_df 

,index,sura,aya,text
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ
3,4,1,4,مَالِكِ يَوْمِ الدِّينِ
4,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...,...
6231,6232,114,2,مَلِكِ النَّاسِ
6232,6233,114,3,إِلَـٰهِ النَّاسِ
6233,6234,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
6234,6235,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ


In [42]:
arabic_words_data

[(2000, 'بِسْمِ', 176),
 (2158, 'السَّمَاءِ', 176),
 (2199, 'وَالسَّمَاءَ', 176),
 (2302, 'سَمَاوَاتٍ', 176),
 (2325, 'الْأَسْمَاءَ', 176),
 (2331, 'بِأَسْمَاءِ', 176),
 (2343, 'بِأَسْمَائِهِمْ', 176),
 (2348, 'السَّمَاوَاتِ', 176),
 (3060, 'اسْمُهُ', 176),
 (4459, 'مُسَمًّى', 176),
 (4737, 'سَمَّيْتُهَا', 176),
 (5199, 'السَّمَاوَاتُ', 176),
 (6604, 'اسْمَ', 176),
 (7223, 'السَّمَاءَ', 176),
 (7740, 'اسْمُ', 176),
 (8264, 'أَسْمَاءٍ', 176),
 (8265, 'سَمَّيْتُمُوهَا', 176),
 (8742, 'الْأَسْمَاءُ', 176),
 (8745, 'أَسْمَائِهِ', 176),
 (10155, 'وَيَاسَمَاءُ', 176),
 (10627, 'أَسْمَاءً', 176),
 (11069, 'سَمُّوهُمْ', 176),
 (11274, 'وَالسَّمَاوَاتُ', 176),
 (12632, 'سَمِيًّا', 176),
 (12872, 'وَالسَّمَاوَاتِ', 176),
 (13711, 'سَمَّاكُمُ', 176),
 (14218, 'السَّمَاءُ', 176),
 (16599, 'سَمَاءٍ', 176),
 (17314, 'الِاسْمُ', 176),
 (17423, 'وَالسَّمَاءِ', 176),
 (17571, 'أَسْمَاءٌ', 176),
 (17575, 'لَيُسَمُّونَ', 176),
 (17576, 'تَسْمِيَةَ', 176),
 (17892, 'بِاسْمِ', 176),
 (18930, 'تُسَمَّى', 17

In [24]:
root_data

[(0, 'س ت ت'),
 (1, 'ن ط ف'),
 (2, 'ص ف ح'),
 (3, 'ه ي ا'),
 (4, 'ن ط ح'),
 (5, 'س ع ر'),
 (6, 'ح ق ق'),
 (7, 'ك ل ح'),
 (8, 'ن ع م'),
 (9, 'ع ن ت'),
 (10, 'س م ر'),
 (11, 'ب ي ض'),
 (12, 'ا ل ت'),
 (13, 'و ل ج'),
 (14, 'ك ا ن ن م ا'),
 (15, 'ص ب ر'),
 (16, 'ه ا ك'),
 (17, 'ض ي ز'),
 (18, 'م ا ل'),
 (19, 'ص د ف'),
 (20, 'ا ذ ا'),
 (21, 'ف و ق'),
 (22, 'ص ي ر'),
 (23, 'ص ل ي'),
 (24, 'ك ه ن'),
 (25, 'خ و ض'),
 (26, 'ذ ع ن'),
 (27, 'د و ن'),
 (28, 'ص ل ب'),
 (29, 'ف ق ع'),
 (30, 'ك ل ف'),
 (31, 'ف ك ك'),
 (32, 'خ ف ف'),
 (33, 'ح و ت'),
 (34, 'س و ف'),
 (35, 'ح ص ر'),
 (36, 'ع ه ن'),
 (37, 'ن ش ط'),
 (38, 'ز ج و'),
 (39, 'ز ك و'),
 (40, 'ا س س'),
 (41, 'ط و ف'),
 (42, 'س ف ح'),
 (43, 'س ف ه'),
 (44, 'ع و ذ'),
 (45, 'ع ط ل'),
 (46, 'ك و ن'),
 (47, 'ز ب ن'),
 (48, 'ب و ا'),
 (49, 'ع ق م'),
 (50, 'ع ج ل'),
 (51, 'ذ ن ب'),
 (52, 'ق ي ض'),
 (53, 'ق م ط ر'),
 (54, 'ف ن ن'),
 (55, 'ز ل م'),
 (56, 'ق س ر'),
 (57, 'ص د ر'),
 (58, 'ح د د'),
 (59, 'ن غ ض'),
 (60, 'ق م ر'),
 (61, 'خ ر ج'),
 (62, 'ع ت

In [25]:
arabic_words_data

[(2000, 'بِسْمِ', 176),
 (2158, 'السَّمَاءِ', 176),
 (2199, 'وَالسَّمَاءَ', 176),
 (2302, 'سَمَاوَاتٍ', 176),
 (2325, 'الْأَسْمَاءَ', 176),
 (2331, 'بِأَسْمَاءِ', 176),
 (2343, 'بِأَسْمَائِهِمْ', 176),
 (2348, 'السَّمَاوَاتِ', 176),
 (3060, 'اسْمُهُ', 176),
 (4459, 'مُسَمًّى', 176),
 (4737, 'سَمَّيْتُهَا', 176),
 (5199, 'السَّمَاوَاتُ', 176),
 (6604, 'اسْمَ', 176),
 (7223, 'السَّمَاءَ', 176),
 (7740, 'اسْمُ', 176),
 (8264, 'أَسْمَاءٍ', 176),
 (8265, 'سَمَّيْتُمُوهَا', 176),
 (8742, 'الْأَسْمَاءُ', 176),
 (8745, 'أَسْمَائِهِ', 176),
 (10155, 'وَيَاسَمَاءُ', 176),
 (10627, 'أَسْمَاءً', 176),
 (11069, 'سَمُّوهُمْ', 176),
 (11274, 'وَالسَّمَاوَاتُ', 176),
 (12632, 'سَمِيًّا', 176),
 (12872, 'وَالسَّمَاوَاتِ', 176),
 (13711, 'سَمَّاكُمُ', 176),
 (14218, 'السَّمَاءُ', 176),
 (16599, 'سَمَاءٍ', 176),
 (17314, 'الِاسْمُ', 176),
 (17423, 'وَالسَّمَاءِ', 176),
 (17571, 'أَسْمَاءٌ', 176),
 (17575, 'لَيُسَمُّونَ', 176),
 (17576, 'تَسْمِيَةَ', 176),
 (17892, 'بِاسْمِ', 176),
 (18930, 'تُسَمَّى', 17

In [23]:
########################################
### GET data for  TEXT TO WORD table ###
########################################

# Create surah and ayah columns 
surah, ayah = [], []
for row in df["ID"]:
    surah.append( row.split(":")[0] ) 
    ayah.append( row.split(":")[1] ) 
df["Surah"] = surah
df["Ayah"] = ayah
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)

# Get the index_ids from the Quran Table  -- need to merge it since different lengths
index_id, count = [], 0
ayahs = df['Ayah'].to_list()
# Essentially generating each ayah = 1 unique id, from 1 - 6000. # Don't need quran table now
for i in range(len(ayahs)):
    if ayahs[i] != ayahs[i-1]:
        count+=1 
        index_id.append(count)
    else:
        index_id.append(count)
    
df['index_id'] = index_id

# Generate our text to words table
TextToWords_df = df.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'inner')
TextToWords_df = TextToWords_df.reset_index()

# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []
for index, row in TextToWords_df.iterrows():
    tup = tuple((row['index_id'], row['primary_key']))
    if tup not in TexttoWord:
        TexttoWord.append( tup )
    else:
        pass


In [2]:

try:

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=PG_USER,
                              password=PG_PASSWORD,
                              host=PG_HOST,
                              port=PG_PORT,
                              database=PG_DB)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

In [3]:
# Create function to get the quran text data 
def get_quran_text_df():
    """ Get the quran text dataframe """
    postgreSQL_select_Query = "select * from quran_text"
    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from table using cursor.fetchall")
    sql_table_result = cursor.fetchall()
    quran_text_df = pd.DataFrame(sql_table_result, columns=['index', 'sura', 'aya','text'])
    return quran_text_df

In [4]:
quran_df = get_quran_text_df()

Selecting rows from table using cursor.fetchall


In [6]:
quran_df["text"][0]

'بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ'

In [7]:
# Extract all of the words in a list from the text 
all_words = list(quran_df['text'].str.split(" "))
unique_words = set().union(*all_words) # Get the unique words

In [9]:
unique_words

{'اتَّبَعَنِ',
 'نَوْمَكُمْ',
 'الْعِجْلَ',
 'فَوَيْلٌ',
 'مُسْلِمَيْنِ',
 'مَاذَا',
 'تُكَذِّبُوا',
 'الرَّحْمَةَ',
 'وَنَزَعْنَا',
 'الشِّرْكَ',
 'وَلْيَتَّقِ',
 'حَاجِزِينَ',
 'أَكْثَرَهُمْ',
 'إِلَـٰهُكُمْ',
 'تَوَلَّيْتُمْ',
 'يُؤْتِيهِ',
 'عَيْنِي',
 'يُعَقِّبْ',
 'تُصْبِحُونَ',
 'شُرَكَاءَ',
 'وَحَلَالًا',
 'فَوْقَهَا',
 'أَأَقْرَرْتُمْ',
 'الْمُلْقِينَ',
 'بِرُسُلٍ',
 'أَعْرِضْ',
 'الْمَاءِ',
 'وَأَيْدِيَكُمْ',
 'جَهُولًا',
 'أَنظُرْ',
 'سُكِّرَتْ',
 'فَأُنَبِّئُكُم',
 'لِّتَعْتَدُوا',
 'يُتْبِعُونَ',
 'لَّبِثْتُمْ',
 'وَيُكَفِّرَ',
 'مَالِكُونَ',
 'بَيَّنَّاهُ',
 'فَالْحُكْمُ',
 'وَقُودُهَا',
 'فَلْيُؤَدِّ',
 'ذِكْرٌ',
 'كَدُعَاءِ',
 'بُيُوتًا',
 'وَّنَصَرُوا',
 'وَقُتِّلُوا',
 'تَرَكْتُ',
 'يَجِدِ',
 'لَّمِنَ',
 'وَبَثَّ',
 'أَضْلَلْتُمْ',
 'يُضْلِلْ',
 'مُخْلِصُونَ',
 'قَلْبٌ',
 'يَتَذَكَّرُ',
 'مِهَادًا',
 'وَنُرِيدُ',
 'فَانتَشِرُوا',
 'خَفِيٍّ',
 'أَيُّكُمْ',
 'وَنِسَاءَنَا',
 'صَاحِبَيِ',
 'ثَيِّبَاتٍ',
 'عَصَوْنِي',
 'خَوْفٌ',
 'وَالطَّارِقِ',
 'وَأَحَاطَتْ',
 'نَفْسٍ',

In [22]:
len(unique_words)

18251

In [26]:
unique_words

{'يَعْصُونَ',
 'دَعًّا',
 'فَيَكِيدُوا',
 'صَبَاحُ',
 'جِمَالَتٌ',
 'وَأَلْقَتْ',
 'تُنبِتُوا',
 'يُحَاجُّونَ',
 'جِئْنَاهُم',
 'جَعَلَا',
 'مَكَرَ',
 'الْجَنَّةُ',
 'مَّنضُودٍ',
 'تُعْلِنُونَ',
 'وَالْجِبِلَّةَ',
 'مُرْسِلِينَ',
 'أَسَأْتُمْ',
 'بِالْقَارِعَةِ',
 'لَبِالْمِرْصَادِ',
 'بِمُعْجِزِينَ',
 'تَلَقَّوْنَهُ',
 'فَاعْدِلُوا',
 'مُّطَهَّرَةٌ',
 'بَلَغْتَ',
 'لَمَسَّكُمْ',
 'شَاكِلَتِهِ',
 'مُتَجَانِفٍ',
 'فَاثْبُتُوا',
 'سُئِلُوا',
 'لِّنَفْتِنَهُمْ',
 'فَكَذَٰلِكَ',
 'فَعَلَيْهِمْ',
 'تَأْخُذُهُ',
 'يُهِنِ',
 'يُحِيطُوا',
 'الْمُلْكِ',
 'عَمِلَتْهُ',
 'قَضَى',
 'زَوْجَهُ',
 'زَاهِقٌ',
 'ظَالِمَةٌ',
 'بِمِثْلِ',
 'أَنفُسُهُمْ',
 'أَخَذْتُ',
 'فَلْيَصُمْهُ',
 'تَعِدُنَا',
 'وَّأَحْسَنُوا',
 'وَذُرِّيَّةً',
 'حَدِيدًا',
 'الْمَلْعُونَةَ',
 'مَتَابًا',
 'سِدْرٍ',
 'وَخَلَقْتَهُ',
 'قَوْمُكَ',
 'أَطْعَمَهُم',
 'آخَرَانِ',
 'بِالْبُخْلِ',
 'فَفَتَحْنَا',
 'فَحَبِطَتْ',
 'وَالْمُنَافِقِينَ',
 'كَلَّمَهُ',
 'عَذَابِي',
 'بِوَاحِدَةٍ',
 'وَالسَّمِيعِ',
 'أَنْهَارًا',
 'وَقُتِّلُوا',
 '

In [ ]:
# Close the DB connection 
cursor.close()
connection.close()
print("PostgreSQL connection is closed")

In [27]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error
import os
from dotenv import load_dotenv

load_dotenv("../.env")

PG_HOST = os.getenv('POSTGRES_HOST')
PG_PORT = os.getenv('POSTGRES_PORT')
PG_PASSWORD = os.getenv('POSTGRES_PASSWORD')
PG_USER = os.getenv('POSTGRES_USER')
PG_DB = os.getenv('POSTGRES_DB')

# Read in the data file that we need to insert
df = pd.read_excel("./Data/Cleaned_Root_letters.xlsx")

# Clean the data right at the start 
df = df.convert_dtypes()
df['ARABIC'] = df['ARABIC'].str.strip()
df['Root_Letters'] = df['Root_Letters'].str.strip()

# Prep Data for RootWord Table 
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

# Turn it into string and get rid of white spaces 
df_roots = df_roots.convert_dtypes()
df_roots['Root_Words'] = df_roots['Root_Words'].str.strip()

# Put the data in the format the db accepts
root_data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    root_data.append((row, arabic_text))

# Prep Data for the ArabicWord Table
arabic_df = df.drop(['ID', 'Transliteration'], axis=1)
arabic_df = arabic_df.drop_duplicates()
arabic_df["primary_key"] = list(range(2000,19623)) 
arabic_df = arabic_df.convert_dtypes()

# Merge the df to get the RootWords PK (called "index")
arabic_words_df = arabic_df.merge(df_roots, how='inner', left_on='Root_Letters', right_on='Root_Words')
arabic_words_df = arabic_words_df.drop(['Root_Letters', 'Root_Words'], axis=1)

# Rename the index column to root_id to make it clear
arabic_words_df.rename(columns = {'index':'root_id'}, inplace = True)

# Put the dataframe into the format for postgres sql insertion 
arabic_words_data = []
for index, row in arabic_words_df.iterrows():
    arabic_words_data.append( (row['primary_key'], row['ARABIC'], row['root_id']) )
    
########################################
### GET data for  TEXT TO WORD table ###
########################################

# Create surah and ayah columns 
surah, ayah = [], []
for row in df["ID"]:
    surah.append( row.split(":")[0] ) 
    ayah.append( row.split(":")[1] ) 
df["Surah"] = surah
df["Ayah"] = ayah
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)

# Get the index_ids from the Quran Table  -- need to merge it since different lengths
index_id, count = [], 0
ayahs = df['Ayah'].to_list()
# Essentially generating each ayah = 1 unique id, from 1 - 6000. # Don't need quran table now
for i in range(len(ayahs)):
    if ayahs[i] != ayahs[i-1]:
        count+=1 
        index_id.append(count)
    else:
        index_id.append(count)
    
df['index_id'] = index_id

# Generate our text to words table
TextToWords_df = df.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'inner')
TextToWords_df = TextToWords_df.reset_index()

# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []
for index, row in TextToWords_df.iterrows():
    tup = tuple((row['index_id'], row['primary_key']))
    if tup not in TexttoWord:
        TexttoWord.append( tup )
    else:
        pass

try:

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=PG_USER,
                              password=PG_PASSWORD,
                              host=PG_HOST,
                              port=PG_PORT,
                              database=PG_DB)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
    
# Create functions to insert root words, arabic words and text to word data   
def insert_root_words(root_words_data):
    """ Insert Root Words Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO RootWord (root_id, root_word) VALUES (%s ,%s)"""
    counter = 0
    for row in root_words_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into rootWord table")
    return counter

def insert_arabic_text(arabic_text_data):
    """ Insert Arabic Text Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO ArabicWord (word_id, word, root) VALUES (%s ,%s, %s)"""
    counter = 0
    for row in arabic_text_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into ArabicWord table")
    return counter

# Create function to get the quran text data 
def get_quran_text_df():
    """ Get the quran text dataframe """
    postgreSQL_select_Query = "select * from quran_text"
    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from table using cursor.fetchall")
    sql_table_result = cursor.fetchall()
    quran_text_df = pd.DataFrame(sql_table_result, columns=['index', 'sura', 'aya','text'])
    return quran_text_df

# Insert the root_words data 
insert_root_words(root_data)

# Insert the arabic words data 
insert_arabic_text(arabic_words_data)

def insert_text_to_word(text_to_word_data):
    """ Insert Text to Word  Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO TextToWord (index_id, word_id) VALUES (%s, %s)"""
    counter = 0
    for row in text_to_word_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into TextToWord table")
    return counter

# Insert text_to_word_df
insert_text_to_word(TexttoWord)

# Close the DB connection 
cursor.close()
connection.close()
print("PostgreSQL connection is closed")

1799 records inserted successfully into rootWord table


UndefinedColumn: column "root" of relation "arabicword" does not exist
LINE 1:  INSERT INTO ArabicWord (word_id, word, root) VALUES (2000 ,...
                                                ^
